In [ ]:
!pip3 install scikit-image --user

In [18]:
import os
import png
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam

from skimage import transform
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
"""
    Preprocess image, normalizing and resizing it

    :param frame: RGBA frame
"""    
def preprocess_image(frame):
    
    # Normalize Pixel Values
    normalized_frame = frame/255.0 - 0.5
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [150,150])
    
    # Create a 3-Channel image
    final_image = np.dstack((preprocessed_frame, preprocessed_frame, preprocessed_frame))
    
    return final_image

def make_labels(labels):
    np_labels = np.zeros((len(labels), 16))
    
    for i in range(len(labels)):
        np_labels[i, labels[i]] = 1
    
    return np_labels

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(16, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
from keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

219062272/219055592 [==============================] - 51s 0us/step


In [24]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(16, activation='softmax'))

In [25]:
conv_base.traiable = False

In [26]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [43]:
train_labels = []
val_labels = []
test_labels = []

train_input = []
val_input = []
test_input = []

folder = "./data/rvl-cdip"
labels_file = open(folder + "/labels/val.txt")

for line in labels_file:
    sp = line.split()
    pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))
    
    val_input.append(pp_img)
    val_labels.append(int(sp[1]))
    
    if len(val_input) >= 100:
        break

x_val = np.asarray(val_input)
y_val = make_labels(val_labels)

labels_file = open(folder + "/labels/train.txt")

for line in labels_file:
    sp = line.split()
    pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))
    
    train_input.append(pp_img)
    train_labels.append(int(sp[1]))
    
    if len(train_input) >= 100:
        x_train = np.asarray(train_input)
        y_train = make_labels(train_labels)
        
        
        model.fit(x_train, y_train,
                  batch_size=10,
                  epochs=1,
                  verbose=1,
                  validation_data=(x_val, y_val))
        
        train_input = []
        train_labels = []

Instructions for updating:
Use tf.cast instead.
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 56s 556ms/step - loss: 2.9126 - acc: 0.1300 - val_loss: 4.6293 - val_acc: 0.0200


KeyboardInterrupt: 